# Web Scraping Gobierno Transparente Chile
##### Exploring wage expenses in the goberment. Public information.

Beautiful soup did not work. The server time outs when you connect through bsoup.

## Trying Selenium

In [18]:
import pandas as pd
import numpy as np
import time
import unicodedata
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

import sys
sys.path.append('./lib/')
#from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [36]:


def createCustomDataFrame():
   
    columns = ['entity', 'department', 'type_contract', 'year', 'month', 'Estamento', 'Apellido paterno', 'Apellido materno', 'Nombres', 'Grado EUS',	'Calificación profesional o formación', 'Cargo', 'Región',	'Asignaciones especiales', 'Unidad monetaria', 'Remuneración Bruta Mensualizada', 'Horas extraordinarias', 'Fecha de inicio', 'Fecha de término', 'Observaciones']
    
    df = pd.DataFrame(columns=columns)
    #df = cleanData(df)
    
    return(df)



def cleanData(df):

# Another way to do the same
#    pd.concat([df[col].astype(str).str.upper() for col in df.columns], axis=1)
    df = df.str.lower()
    df = df.str.replace('á', 'a')
    df = df.str.replace('é', 'e')
    df = df.str.replace('í', 'i')
    df = df.str.replace('ó', 'o')
    df = df.str.replace('ú', 'u')        

    df['Remuneración Bruta Mensualizada'] = df['Remuneración Bruta Mensualizada'].replace(to_replace = "\.+", value="", regex=True)

    return(df)


def getGovernmentData(output_file, url, browser, numEnt):

    gov_data = entity_data = createCustomDataFrame()
    browser.get(url)

    url_list = []
    entities = browser.find_elements_by_class_name("primaryCat")

    counter = 0
    for e in entities:
        entity_url = e.get_attribute("href")
        url_list.append(entity_url)

#    print('Entities:')
#    print(url_list)
    for url in tqdm_notebook(url_list):
        getEntityData(output_file, url, browser)

def getEntityData(output_file, url, browser):
    browser.get(url)
    entity_data = createCustomDataFrame()
    
    url_list =[]
    departments = browser.find_elements_by_class_name("primaryCat")

    for d in departments:
        department_url = d.get_attribute("href")
        url_list.append(department_url)

#    print('Departments:')
#    print(url_list)
    for url in tqdm_notebook(url_list):
        getDepartmentData(output_file, url, browser)

def getDepartmentData(output_file, url, browser):
    department_data = createCustomDataFrame()
    
    type_contract = ['per_planta', 'per_contrata']
    url_list = []

    for t in type_contract:
        dept_contract_link = url + "/" + t
        browser.get(dept_contract_link)
#        try:
        div_years = browser.find_element_by_class_name("linksIntermedios")
        unordered_list = div_years.find_element_by_tag_name("ul")
        years_links_list = unordered_list.find_elements_by_tag_name("li")
        for l in years_links_list:
            link_anchor = l.find_element_by_tag_name("a")
            year_url = link_anchor.get_attribute("href")
            url_list.append(year_url)

#    print('Years:')
#    print(url_list)
    for url in tqdm_notebook(url_list):
        getYearData(output_file, url, browser)

        
        
def getYearData(output_file, url, browser):
    
    # Check if we still have to dive down into the months
    div_months = False
    try:
        div_months = browser.find_element_by_class_name("linksIntermedios")
    except NoSuchElementException:
        div_months = False

    url_list = []
    
    year_data = createCustomDataFrame()
    
    if div_months:

        unordered_list = div_months.find_element_by_tag_name('ul')
        months_links_list = unordered_list.find_elements_by_tag_name('li')

        for m in months_links_list:
            month_link = m.find_element_by_tag_name('a')
            month_url = month_link.get_attribute('href')
            url_list.append(month_url)

    else:
        url_list.append(url)
    
    for url in url_list:
        print(url)
    
    for url in tqdm_notebook(url_list):  # debug purposes
        getDatainPage(output_file, url, browser)

    

def getDatainPage(output_file, url, browser):

    browser.get(url)

    table_links = []
    table_links.append(url)

    try:
        pagination = browser.find_element_by_class_name("pagination")
        pages = pagination.find_elements_by_tag_name("li")

        for page in pages:
            try:
                link_element = page.find_element_by_tag_name("a")
                link = link_element.get_attribute("href")
                table_links.append(link)
            except NoSuchElementException:
                pass

        # Remove the last element, its the back arrow
        del table_links[-1]

    except:
        pass

    total_tables = len(table_links)

    # Loop through all the pages and record data
    for count, i in enumerate(table_links):
        try:
            if count == 0:
                getTableData(output_file, i, browser)
            else:
                getTableData(output_file, i, browser)

        except:
            pass

def getTableData(output_file, url, browser):
    browser.get(url)

    #######
    ### 1 Get table metadata from the breadcrumb (Year, Department, Kind of contract, ...)
    #######

    try:
        table_location_data = browser.find_element_by_class_name("breadcrumb")
        breadcrumb_items = table_location_data.find_elements_by_tag_name("li")
        num_breadcrumbs = len(breadcrumb_items)

    except NoSuchElementException:
        print ('could not get breadcrumbs from ' + url)
        f = open('./output/log_not_opened.txt', 'a')
        f.write(url + +',' + 'Error Reading Page' + "\n");
        f.close()

    entity = breadcrumb_items[1].text
    department = breadcrumb_items[2].text
    type_contract = breadcrumb_items[3].text
    year = breadcrumb_items[4].text
    if num_breadcrumbs == 6:
        month = breadcrumb_items[5]
    else:
        month = 'allyear'

        #######
        ### 2 Get Data of table
        #######

    try:
        data = browser.find_elements_by_tag_name("table")
        
    except NoSuchElementException:
        print ('could not get table from ' + url)
        f = open('./output/log_not_opened.txt', 'a')
        f.write(url + +',' + 'Error Reading Page' + "\n");
        f.close()

    for i in data:

        # Headers
        head = i.find_elements_by_tag_name("thead")
        for j in head:
            header_row = j.find_elements_by_tag_name("th")

            # Get length and list of headers
            ncol = len(header_row)

            headers = list()
            # Add table location variables
            headers_add = ['entity', 'department', 'type_contract',
                          'year', 'month']

            for h in headers_add:
                headers.append(h)
            # Now add actual data row values
            for k in header_row:
                headers.append(k.text)

            headers.append('url')

        # Prepare data frame
        df = pd.DataFrame(columns=headers)

        # Actual Data
        table_data = i.find_elements_by_tag_name("tbody")

        for j in table_data:
            data_row = j.find_elements_by_tag_name("tr")

            # Get length and list of data rows
            nrow = len(data_row)      
            extracted_rows = 0
            master_list = list()
            for count, k in enumerate(data_row):
#                    t1 = time.time()
#                    print('Starting to parse row' + str(count) + ' of ' + str(nrow) + '\n')

                data_element = k.find_elements_by_tag_name("td")

                # Process only if there is data in the row
                if len(data_element) != 0:
                    actual_record = list()

                    # Add table location variables
                    actual_record.append(entity)
                    actual_record.append(department)
                    actual_record.append(type_contract)
                    actual_record.append(year)
                    actual_record.append(month)

                    # Populate list with actual record

                    for l in data_element:
                        actual_record.append(l.text)

                    # Add url
                    actual_record.append(url)

                    master_list.append(actual_record)
                    extracted_rows += 1
#                        t2 = time.time()
#                        print('Finished ' + str(t2 - t1))

    df = pd.DataFrame(master_list, columns = headers)


    try:
        f = open(output_file, 'a')
        df.to_csv(f, header=False)    
        f.close()
        
        a = open('./output/log_opened.txt', 'a')
        a.write(url + ',' + str( time.time()  ) + "\n");
        a.close()

    except:
        print ('could not get data from ' + url)
        g = open('./output/log_error.txt', 'a')
        g.write(url + ',' + 'Error writing' + "\n");
        g.close()

#        return(None)

    
        

In [ ]:
output = './output/scraped_data_2.csv'

url = 'http://www.gobiernotransparentechile.cl/directorio/entidad'
browser = webdriver.Firefox()

df = getGovernmentData(output, url, browser, 1)

browser.quit()

http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2016
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2015
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2014
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2013
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2012
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2011
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2010
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2009
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2008
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2007
http://www.gobiernotransparentechile.cl/directorio/entidad/1/1/per_contrata/Ao-2006

